In [1]:
from pathlib import Path
#import warnings
import scanpy as sc
import scib 
import numpy as np
import pandas as pd
import sys 

import scgpt as scg
import matplotlib.pyplot as plt

plt.style.context('default')
#warnings.simplefilter('ignore', ResourceWarning)
import warnings
warnings.filterwarnings('ignore', category=UserWarning)






c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\scgpt\model\model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\scgpt\model\multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
c:\Users\annel\anaconda3\envs\scgpt_py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys

repo_dir = Path.cwd().parent.absolute()
sys.path.append(str(repo_dir))

from src.utils import setup_data_dir
setup_data_dir()
data_dir = repo_dir / "data"

print(repo_dir, data_dir)



c:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT c:\Users\annel\OneDrive\Documenten\Machine Learning\scGPT\data


In [3]:
file_path = data_dir / "human_pancreas_norm_complexBatch.h5ad" 
adata = sc.read_h5ad(file_path) 






In [4]:
preprocessor = scg.preprocess.Preprocessor(
    use_key = "X",
    filter_gene_by_counts = 10,
    filter_cell_by_counts = 10,
    normalize_total = 1e4,
    result_normed_key = "X_normed",
    log1p = True,
    result_log1p_key = "X_log1p",
    subset_hvg = True, #calculate hvg 
    hvg_use_key = "X_log1p", #which layer to use for hvg selection
    hvg_flavor = "seurat_v3",
    hvg_subset = False,
    binning = 10,
    result_binned_key = "X_binned",
)

TypeError: __init__() got an unexpected keyword argument 'hvg_subset'

In [14]:
preprocessor(adata, batch_key = "tech")


scGPT - INFO - Filtering genes by counts ...
scGPT - INFO - Filtering cells by counts ...
scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Log1p transforming ...
scGPT - WARNING - The input data seems to be already log1p transformed. Set `log1p=False` to avoid double log1p transform.
scGPT - INFO - Subsetting highly variable genes ...
scGPT - INFO - Binning data ...
scGPT - WARNING - The input data contains all zero rows. Please make sure this is expected. You can use the `filter_cell_by_counts` arg to filter out all zero rows.
scGPT - WARNING - The input data contains all zero rows. Please make sure this is expected. You can use the `filter_cell_by_counts` arg to filter out all zero rows.
scGPT - WARNING - The input data contains all zero rows. Please make sure this is expected. You can use the `filter_cell_by_counts` arg to filter out all zero rows.
scGPT - WARNING - The input data contains all zero rows. Please make sure this is expected. You can use the `filter_cell_by_co

In [19]:
print (adata.layers["X_normed"])
print (adata.layers["X_log1p"])
print (adata.layers["X_binned"])
print (adata.X)
print (adata)







[[2.0193782]
 [1.5303893]
 [1.8410705]
 ...
 [1.1969761]
 [1.4378303]
 [1.5801585]]
[[2.0193782]
 [1.5303893]
 [1.8410705]
 ...
 [1.1969761]
 [1.4378303]
 [1.5801585]]
[[6]
 [9]
 [7]
 ...
 [5]
 [7]
 [1]]
[[2.2416177]
 [1.7432532]
 [2.0180078]
 ...
 [1.2445253]
 [1.4739832]
 [1.7131797]]
AnnData object with n_obs × n_vars = 16382 × 1
    obs: 'tech', 'celltype', 'size_factors', 'n_counts'
    var: 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'log1p', 'hvg'
    obsm: 'bin_edges'
    layers: 'counts', 'X_normed', 'X_log1p', 'X_binned'


In [7]:
#Example file for human pancreas data

import scanpy as sc
import scipy.sparse

from anndata._warnings import OldFormatWarning

# Suppress AnnData old format warnings
warnings.filterwarnings('ignore', category=OldFormatWarning)



def load_data(file_path, n_top_genes = 1800, hvg = True, show_info=True): 

    adata = sc.read_h5ad(file_path) 
    gene_col = "gene_name"
    cell_type_key = "celltype" #where the cell names are stored
    batch_key = "tech" #show from different methods 
    #N_HVG = 1800 #number of high variable genes 

    if hvg and 'highly_variable' not in adata.var.columns:
        sc.pp.highly_variable_genes(adata, n_top_genes = n_top_genes, flavor='seurat_v3')
        adata = adata[:, adata.var['highly_variable']]
    
    # Check if data is normalized and normalize if needed
    if adata.X.max() > 10:  # Simple heuristic to check if data needs normalization
        print("Data appears unnormalized. Performing normalization...")
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
    else:
        print("Data appears to be already normalized")

    if show_info:
        # Look at the structure and content
        print("AnnData object:", adata)
        print("\nShape:", adata.shape)  # (n_obs × n_vars)


        # Check if X is sparse
        print("\nX storage type:", type(adata.X))
        if scipy.sparse.issparse(adata.X):
            print("Sparse matrix details:")
            print('It is a sparse matrix')
            print("- data shape:", adata.X.data.shape)
            print("- indices shape:", adata.X.indices.shape)
            print("- indptr shape:", adata.X.indptr.shape)

        # Look at obs and var contents
        print("\nContents of obs:", adata.obs.columns.tolist())
        print("\nContents of var:", adata.var.columns.tolist())

        # print some metadata
        print("\nMetadata:")
        print(adata.uns)
        print("\nLayers:")
        print(adata.layers)
    


    #save function 

    return adata



example_adata = load_data(file_path, n_top_genes = 1800, hvg = True, show_info=False)



Data appears to be already normalized


In [4]:
print(example_adata)

View of AnnData object with n_obs × n_vars = 16382 × 1800
    obs: 'tech', 'celltype', 'size_factors'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'


In [5]:
import os 

def save_adata(adata, file_path, hvg = True):
    if hvg and 'highly_variable' in adata.var.columns:
        print("Saving HVG data")
        adata_hvg= adata[:, adata.var['highly_variable']]
        save_dir = os.path.join(os.path.dirname(file_path), 'save')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        base_name = os.path.basename(file_path)
        new_name = base_name.replace('.h5ad', '_hvg.h5ad')
        file_path = os.path.join(save_dir, new_name)
        adata_hvg.write_h5ad(file_path)
    else:
        adata.write_h5ad(file_path)




save_adata(example_adata, file_path)


Saving HVG data
